In [1]:
import pandas as pd
import numpy as np

#### Anlegen eines Dummy-DataFrames

In [2]:
df = pd.DataFrame({"col1":['a','b','b', np.nan], "col2":[0,1,np.nan,2]})

In [3]:
df.head()

,col1,col2
0,a,0.0
1,b,1.0
2,b,NaN
3,NaN,2.0


In [4]:
df.dtypes

col1     object
col2    float64
dtype: object

In [5]:
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator

from pandas.api.types import is_numeric_dtype

#Imputer für die numerischen Spalten eines DataFrame
class NumericImputer(TransformerMixin, BaseEstimator):
    def __init__(self):
        self.impute_values = {} #zur Speicherung der einzusetzenden Werte
                
    def fit(self, X):
        for col in X.columns:
            if is_numeric_dtype(X[col].dtype):
                self.impute_values[col] = X[col].mean() #Speicherung der Mittelwerte der numerischen Spalten
        return self
            
    def transform(self, X):
        X_transformed = X.copy()
        
        for col in self.impute_values.keys():
            if col in X_transformed.columns:
                X_transformed[col] = X_transformed[col].fillna(self.impute_values[col])
                
        return X_transformed

In [6]:
imp_num = NumericImputer()
imp_num.fit_transform(df)

,col1,col2
0,a,0.0
1,b,1.0
2,b,1.0
3,NaN,2.0


In [7]:
imp_num.impute_values

{'col2': 1.0}

#### Anwendung von ColumnTransformers

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

Mit Hilfe eines ColumnTransformers wird nun der `NumericImputer` auf die (numerische) Spalte `col2` und ein <a href="https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html">SimpleImputer</a> auf die Spalte `col1` angewendet:

In [9]:
transformer = ColumnTransformer(
    [("imp_categorical", SimpleImputer(strategy='most_frequent'),['col1']),
     ("imp_numeric", NumericImputer(),['col2'])]
)


In [10]:
transformer.fit_transform(df)

array([['a', 0.0],
       ['b', 1.0],
       ['b', 1.0],
       ['b', 2.0]], dtype=object)

Der Zugriff auf die einzelnen Transformers ist über das Attribut `named_transformers_` möglich:

In [11]:
transformer.named_transformers_['imp_numeric'].impute_values

{'col2': 1.0}

#### Hintereinanderschalten von Transformers in einer Pipeline

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

#1. Schritt: Impute, 2. Schritt: Standardisierung
pipe_preprocessing_num = Pipeline([('imp_num', NumericImputer()), ('scaler', StandardScaler())])
pipe_preprocessing_cat = Pipeline([('imp_cat', SimpleImputer(strategy='most_frequent')), ('ohe', OneHotEncoder())])

Die Pipeline kann in einem ColumnTransformer verwendet werden:

In [17]:
#Wende auf die kategorischen Spalten den SimpleImputer mit strategy="most_frequent" an, auf die numerischen die Pipeline bestehend aus
#dem NumericImputer und dem StandardScaler:

ct = ColumnTransformer(
    [("preprocessor_cat", pipe_preprocessing_cat,['col1']),
     ("preprocessor_num", pipe_preprocessing_num,['col2'])]
)

In [14]:
ct.fit_transform(df)

array([[ 1.        ,  0.        , -1.41421356],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.        ,  1.        ,  1.41421356]])

Zugriff auf preprocessor_num:

In [15]:
ct.named_transformers_['preprocessor_num'].transform(df[['col2']])

array([[-1.41421356],
       [ 0.        ],
       [ 0.        ],
       [ 1.41421356]])

Zugriff auf StandardScaler:

In [16]:
ct.named_transformers_['preprocessor_cat'].named_steps['imp_cat'].transform(df[['col1']])

array([['a'],
       ['b'],
       ['b'],
       ['b']], dtype=object)